In [2]:
%run setup_levy_model_posner.py

2024-06-10 10:25:12.172051: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 10:25:12.173414: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 10:25:12.193685: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 10:25:12.193705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 10:25:12.194198: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [24]:
f = open("training_data_posner.obj","rb")
train_data  = pickle.load(f)
f.close()

In [25]:
history = trainer.train_offline(
    simulations_dict = train_data, 
    epochs = 100, 
    batch_size = 16
)

Training epoch 1:   0%|          | 0/6250 [00:00<?, ?it/s]